## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

from dotenv import load_dotenv

Purpose: Imports load_dotenv to load environment variables from a .env file (like API keys).

Meaning: .env is a hidden file containing sensitive credentials—this avoids hard-coding secrets in notebooks.

import os

Purpose: Python’s standard os module lets you interact with the operating system.

Usage in your notebook: You’ll later call os.getenv('GROQ_API_KEY') to retrieve the key loaded by load_dotenv().

🗣 Interview Tip:

“The os module gives access to environment variables and file-system utilities.
Here I used os.getenv() to fetch API keys securely.”

groq_api_key = os.getenv("GROQ_API_KEY")

Purpose: Fetches your Groq API key from environment variables.

os.getenv() → retrieves the value of an environment variable.

If the key doesn’t exist, it returns None

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.3-70b-versatile",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B7AD0F9E80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B7AD0F9FD0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

model = ChatGroq(model="llama-3.3-70b-versatile", groq_api_key=groq_api_key)

Purpose: Initializes the Groq LLM client using your API key.

Parameters:

model → specifies which Groq model to use (e.g., "Gemma2-9b-It").

groq_api_key → authenticates your access to Groq’s inference service.

##“I used ChatGroq from LangChain to connect with a locally hosted LLM like Gemma or Mixtral. It provides a similar interface to OpenAI’s chat models but runs on Groq hardware, offering faster inference and no dependency on API keys.”


from langchain.prompts import PromptTemplate

Purpose: Lets you create a structured prompt—a reusable template with variables.

What it does: Instead of hard-coding text, you define placeholders (like {question} or {context}) that LangChain fills dynamically before sending the prompt to the model.

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Keerthi and I am a Chief AI Engineer")])

AIMessage(content="Nice to meet you, Keerthi! It's great to connect with a Chief AI Engineer like yourself. That's a fascinating role, and I'm sure you're doing some cutting-edge work in the field of artificial intelligence. What specific areas of AI are you currently focused on, and what projects are you most excited about?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 51, 'total_tokens': 119, 'completion_time': 0.167152673, 'prompt_time': 0.002416432, 'queue_time': 0.054376238, 'total_time': 0.169569105}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_fb4860a75b', 'finish_reason': 'stop', 'logprobs': None}, id='run--3fae0e22-ae42-4b2c-afe5-509c283ec54f-0', usage_metadata={'input_tokens': 51, 'output_tokens': 68, 'total_tokens': 119})

#### from langchain.chains import LLMChain
Purpose: Imports LLMChain, LangChain’s basic building block that links a prompt template → LLM → output.

Meaning:

A Chain automates the sequence: input → prompt formatting → model inference → output parsing.

You can combine multiple chains to form complex workflows (e.g., retrieval + reasoning + generation).

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Keerthi and I am a Chief AI Engineer"),
        AIMessage(content="Hello Keerthi! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="Your name is Keerthi, and you're a Chief AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 123, 'total_tokens': 140, 'completion_time': 0.033183333, 'prompt_time': 0.006252556, 'queue_time': 0.057520314, 'total_time': 0.039435889}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_fb4860a75b', 'finish_reason': 'stop', 'logprobs': None}, id='run--7baf8a0c-e625-4571-a289-414266cd9168-0', usage_metadata={'input_tokens': 123, 'output_tokens': 17, 'total_tokens': 140})

from langchain.memory import ConversationBufferMemory

Purpose: Adds chat memory so the model remembers previous interactions.

How it works: Stores user/AI messages in a buffer and appends them to the next prompt, giving conversational context.

🗣 Interview Tip:

“I integrated ConversationBufferMemory so the chatbot could maintain context across turns.
This buffer stores dialogue history, preventing repetition and ensuring continuity in multi-turn conversations.”

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [7]:
!pip install langchain_community

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:  # Session Id is created ,The return
    #type of this function is BaseChatMessageHistory,Session ID is used to distinguish one chat session with other
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

#Whenever i give a session ID ,it should be ble to check whether the session id present in this dictionary or not
#If session id is present ,we are going to get the entire chat message history from that.
# Two things get_session_history will give you a response type of baseChatMessageHistory and it is an
#abstract class for storing the chat message history
#chat message history is an in-memory implementation of the chat message history stores message in memory list
#whenver i get the session id it go and check in this particular dictionary whether it is available or not
#If it is available it will go and pick up the entire chat message w.r.t whatever questions we have asked w.r.t the llm model for that session id
#RunnableWithMessageHistory:we can interact with our llm model based on chat history

In [9]:
config={"configurable":{"session_id":"chat1"}}

In [10]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Keerthi and I am a Data Scientist")],
    config=config
)

In [9]:
response.content

"Nice to meet you, Keerthi! As a Data Scientist, you must be working with data to extract insights and knowledge. That's a fascinating field. What kind of projects or industries have you been working on? Are you specializing in any particular area like machine learning, natural language processing, or computer vision? I'm here to chat and help with any questions or topics you'd like to discuss!"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content="Your name is Keerthi, and you're a Data Scientist.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 147, 'total_tokens': 163, 'completion_time': 0.03125782, 'prompt_time': 0.008083835, 'queue_time': 0.068368575, 'total_time': 0.039341655}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_eff2255fd7', 'finish_reason': 'stop', 'logprobs': None}, id='run--57f3d1e2-681e-4218-9ee0-78b6e69da745-0', usage_metadata={'input_tokens': 147, 'output_tokens': 16, 'total_tokens': 163})

In [11]:
with_message_history.invoke(
    [HumanMessage(content="What's my age?")],
    config=config,
)

AIMessage(content="I don't have any information about your age, Keerthi. We just started chatting, and you only mentioned your name and profession as a Data Scientist. If you'd like to share your age, I'd be happy to know, but it's not necessary!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 160, 'total_tokens': 216, 'completion_time': 0.160371647, 'prompt_time': 0.007506938, 'queue_time': 0.067960562, 'total_time': 0.167878585}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_fb4860a75b', 'finish_reason': 'stop', 'logprobs': None}, id='run--8492aaf4-85f6-4fa0-b1a9-873012e72ced-0', usage_metadata={'input_tokens': 160, 'output_tokens': 56, 'total_tokens': 216})

In [ ]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

# Session Id: The particular chat history is belong to particular session id if i change the session ib  AI do not know  previou one or it is new one
# config={"configurable":{"session_id":"chat1"}}  
#config1={"configurable":{"session_id":"chat2"}} 
#both are different both ae independent to each other

"I don't know your name. I'm a large language model, I don't have the ability to recall personal information about individuals, and our conversation just started. If you'd like to share your name, I'd be happy to chat with you and use it in our conversation!"

In [ ]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Nice to meet you, John! It's great to have a name to associate with our conversation. How's your day going so far? Is there something on your mind that you'd like to talk about, or are you just looking for some casual chat? I'm all ears (or rather, all text)!"

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'I remember! Your name is John. We just established that a moment ago!'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [16]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [17]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Keerthi")]})

AIMessage(content="Hello Keerthi! It's nice to meet you. How can I assist you today? Do you have any questions or topics you'd like to discuss? I'm here to help.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 60, 'total_tokens': 100, 'completion_time': 0.081494146, 'prompt_time': 0.00263303, 'queue_time': 0.05722754, 'total_time': 0.084127176}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_fb4860a75b', 'finish_reason': 'stop', 'logprobs': None}, id='run--cb353f66-838a-49c8-b0f9-085d6566988f-0', usage_metadata={'input_tokens': 60, 'output_tokens': 40, 'total_tokens': 100})

In [21]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [22]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Keerthi")],
    config=config
)

response

AIMessage(content="Nice to meet you, Keerthi! I'll start fresh with you. It seems I had a previous conversation with someone named Krish, but that's all cleared up now. How can I assist you today, Keerthi? Is there something on your mind that you'd like to talk about or ask?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 224, 'total_tokens': 290, 'completion_time': 0.241648395, 'prompt_time': 0.014089063, 'queue_time': 0.058940737, 'total_time': 0.255737458}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_0ca3f8c386', 'finish_reason': 'stop', 'logprobs': None}, id='run--ad759553-1e2c-4eab-902f-1710ddf41ce9-0', usage_metadata={'input_tokens': 224, 'output_tokens': 66, 'total_tokens': 290})

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Keerthi.'

In [19]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [26]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Keerthi")],"language":"kannada"})
response.content

'ನಮಸ್ಕಾರ ಕೀರ್ತಿ, ನಿಮಗೆ ಏನಾದರು ಸಹಾಯ ಮಾಡಲಿ ಅಂತ ಆಲೋಚಿಸಿದ್ರಿ?'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [27]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [30]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Keerthi Shetty")],"language":"kannada"},
    config=config
)
repsonse.content

'ನಮಸ್ಕಾರ ಕೀರ್ತಿ ಶೆಟ್ಟಿ, ನಿಮಗೆ ಸಹಾಯ ಮಾಡಲು ನಾನು ಇಲ್ಲಿದ್ದೇನೆ. ನಿಮಗೆ ಏನಾದರು ಸಹಾಯ ಬೇಕಾ?'

In [32]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "kannada"},
    config=config,
)

In [33]:
response.content

'ನಿಮ್ಮ ಹೆಸರು ಕೀರ್ತಿ ಶೆಟ್ಟಿ.'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [54]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=80,
    strategy="last",
    token_counter=model,#used for count the tokens
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [55]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

'You like vanilla ice cream!'

In [47]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked what 2 + 2 is. The answer is 4.'

In [48]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [49]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'Your name is Bob!'

In [50]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked what 2 + 2 is.'

In [ ]:
#trimmer: this function will trim or delete the conversation for example
# from langchain_core.messages import SystemMessage,trim_messages
# trimmer=trim_messages(
#     max_tokens=45
#     strategy="last",
#     token_counter=model,#used for count the tokens
#     include_system=True,
#     allow_partial=False,
#     start_on="human"
# )
# messages = [
#     SystemMessage(content="you're a good assistant"),
#     HumanMessage(content="hi! I'm bob"),
#     AIMessage(content="hi!"),
#     HumanMessage(content="I like vanilla ice cream"),
#     AIMessage(content="nice"),
#     HumanMessage(content="whats 2 + 2"),
#     AIMessage(content="4"),
#     HumanMessage(content="thanks"),
#     AIMessage(content="no problem!"),
#     HumanMessage(content="having fun?"),
#     AIMessage(content="yes!"),
# ]
# trimmer.invoke(messages)
# In this example we set the max_tokens=45 means that trimmer trims to maximum 20 token so the it has([SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 #((HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 #(AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]))
 #only this much sentence so if i ask for which is my favourte ice cream it should not give vanilla instead
 #it says i do not know even thouggh we mentioned in the conversation beacause we set max_tokens=45 so the 
 #conversation get trimmed

In [57]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,#used for count the tokens
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [58]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"I don't have any information about your preferences, including your favorite ice cream flavor. I'm a text-based AI assistant, and our conversation just started, so I don't have any prior knowledge about you. Would you like to share your favorite ice cream flavor with me?"